In [4]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=ae9555eb405c813d164c2cc4f8f27d7d0affc2bbc01b7287140147d5ab534561
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [5]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 2.9 MB/s 
     |████████████████████████████████| 88 kB 6.5 MB/s 
     |████████████████████████████████| 5.2 MB 44.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 5.2 MB/s 
     |████████████████████████████████| 4.7 MB 33.8 MB/s 
     |████████████████████████████████| 1.2 MB 48.7 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
     |████████████████████████████████| 636 kB 67.7 MB/s 
     |████████████████████████████████| 6.6 MB 50.7 MB/s 
     |████████████████████████████████| 1.1 MB 65.5 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2340316 sha256=13b3082acc91419c2e6499d729762a0979712c55345e784b8e7d6e161af032cd
  Stored in directory: /root/.cache

In [6]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
import pandas as pd
df = pd.read_csv("/content/gdrive/My Drive/Enron Project/Enron_mail_preprocessed-Oct-Nov-Snapshot.csv")
#df['narr_answers'] = df['narr_answers'].str.lower()
df.head()

,mail_text
0,phillip there are number alternative systems t...
1,phillip could you please favor would like read...
2,phillip pursuant your request please see the a...
3,mmpdoc phillip enclosed please find the first ...
4,mmpreddoc phillip enclosed please find blackli...


In [30]:
import re
from stop_words import get_stop_words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
stop_words = list(get_stop_words('en'))

#print((set(list(STOPWORDS)) | set(list(stop_words))))
stopwords = list(set(list(STOPWORDS)) | set(list(stop_words)))
stopwords = [re.sub('[^a-zA-Z0-9]+', '', word) for word in stopwords]

stopwords = stopwords + ['fyi', 'ily', 'britt', 'fyijr', 'fyyi', 'gwf', 'dear', 'hourahead', 'let', 'know', 'make', 'see', 'soon', 'hope', 'yes', 'incredibleforget',
                         'betweeneveryone', 'enron', 'lexisnexis', 'pmeb', 'attend', 'attach', 'message', 'please', 'file']

df['clean_mail_text'] = df['mail_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [31]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()

df['verblem_mail_text'] = df['clean_mail_text'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="v") for word in row.split()))
df['nounlem_mail_text'] = df['verblem_mail_text'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="n") for word in row.split()))
df['adjlem_mail_text'] = df['nounlem_mail_text'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="a") for word in row.split()))
df['advlem_mail_text'] = df['adjlem_mail_text'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="r") for word in row.split()))

df['advlem_mail_text'] = df['advlem_mail_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [32]:
print(len(list(df['advlem_mail_text'])))
doc = []
for i in list(df['advlem_mail_text']):
  if i != "":
    doc.append(i)

print(len(doc))

17540
17223


In [33]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(doc, show_progress_bar=True)

Batches:   0%|          | 0/539 [00:00<?, ?it/s]

In [34]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True, n_gram_range=(2, 2))
topics, probs = topic_model.fit_transform(doc, embeddings)

2022-08-05 23:16:31,211 - BERTopic - Reduced dimensionality
2022-08-05 23:20:07,605 - BERTopic - Clustered reduced embeddings


In [35]:
freq = topic_model.get_topic_info(10); 
freq

,Topic,Count,Name
0,10,78,10_new password_user name_choose new_name pass...


In [36]:
topic_model.get_topic(0)

[('continue start', 0.14506231153231933),
 ('detect log', 0.12366172047869416),
 ('error retrieve', 0.1126176454938224),
 ('ancillary schedule', 0.1126176454938224),
 ('award variance', 0.1126176454938224),
 ('data process', 0.1126176454938224),
 ('hour ancillary', 0.1126176454938224),
 ('retrieve price', 0.1126176454938224),
 ('schedule award', 0.1126176454938224),
 ('schedulestxt error', 0.1125281167514379)]

In [37]:
import matplotlib.pyplot as plt
#plt.rcParams["figure.figsize"] = (20,3)
topic_model.visualize_distribution(probs[10], min_probability=0.000, width = 700, height = 700)

In [43]:
topic_model.visualize_hierarchy(top_n_topics=10)

In [39]:
topic_model.visualize_barchart(top_n_topics=10, n_words = 10, width = 300, height = 300)

In [42]:
topic_model.visualize_heatmap(top_n_topics=10, width=800, height=800)